# Contents:

*   [Libraries](#libs)
*   [Importing Data](#imp)
*   [Feature Extraction and Preprocessing](#prep)
*   [Similarity Function](#sim)
*   [Interacting with user](#user)
*   [Recommendation](#rec)








<a name ='libs'></a>
# Libraries:

In [1]:
import pandas as pd
import numpy as np
import difflib

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

In [2]:
cd /content/drive/MyDrive/Movies Recommender

/content/drive/MyDrive/Movies Recommender


<a name = 'data'></a>
# Data:

In [3]:
df = pd.read_csv('movies.csv')
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
df.shape

(4803, 24)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [6]:
df.isna().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

<a name = 'prep'></a>

# Feature Extraction and Preprocessing:

In [7]:
selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director']

In [8]:
# make copy of data
movies = df[selected_features].copy()
movies.head(1)

,genres,keywords,tagline,cast,director
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron


## Dealing with missing values:

In [9]:
# replace null values with empty string
movies = movies.fillna('')
# check
movies.isna().sum()

genres      0
keywords    0
tagline     0
cast        0
director    0
dtype: int64

## combining Features together

In [10]:
# combine all selected features into one column
combined_features = movies.apply(' '.join, axis = 1)
#check
combined_features

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

## Vectroizing Features:

In [11]:
# using Tfidf vectorizer to represent the combined features in numerical form
vectorizer = TfidfVectorizer()

In [12]:
vectorized_features = vectorizer.fit_transform(combined_features)
#check
vectorized_features

<4803x17318 sparse matrix of type '<class 'numpy.float64'>'
	with 124266 stored elements in Compressed Sparse Row format>

In [13]:
print(vectorized_features)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

# Similarity Score Calculation:

In [14]:
# as we have a normalized representation, the linear kernel will calculate the cosine similarity
similarity_scores = linear_kernel(vectorized_features)
#check
similarity_scores[:5]

array([[1.        , 0.07219487, 0.037733  , ..., 0.        , 0.        ,
        0.        ],
       [0.07219487, 1.        , 0.03281499, ..., 0.03575545, 0.        ,
        0.        ],
       [0.037733  , 0.03281499, 1.        , ..., 0.        , 0.05389661,
        0.        ],
       [0.0125202 , 0.0207415 , 0.05179498, ..., 0.00363016, 0.        ,
        0.        ],
       [0.10702574, 0.03305463, 0.05772601, ..., 0.        , 0.0234792 ,
        0.        ]])

In [15]:
similarity_scores.shape

(4803, 4803)

In [32]:
#save it
np.save('sim_scores.npy', similarity_scores)
s2 = np.load('sim_scores.npy')
#check
s2[:5]

array([[1.        , 0.07219487, 0.037733  , ..., 0.        , 0.        ,
        0.        ],
       [0.07219487, 1.        , 0.03281499, ..., 0.03575545, 0.        ,
        0.        ],
       [0.037733  , 0.03281499, 1.        , ..., 0.        , 0.05389661,
        0.        ],
       [0.0125202 , 0.0207415 , 0.05179498, ..., 0.00363016, 0.        ,
        0.        ],
       [0.10702574, 0.03305463, 0.05772601, ..., 0.        , 0.0234792 ,
        0.        ]])

## List all movies names:

In [16]:
movies_names = df['title'].tolist()
# check
movies_names

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

<a name = 'user'></a>

# Collecting History from User:

In [17]:
movie_name = input('Enter your favourite movie name: ')

Enter your favourite movie name: iron man


## Getting the closest match to user input:

In [18]:
closest_match = difflib.get_close_matches(movie_name, movies_names)
# check
closest_match

['Iron Man', 'Iron Man 3', 'Iron Man 2']

In [19]:
# get the first match
f_match = closest_match[0]

In [20]:
# get index of the first match
f_match_idx = df[df.title == f_match]['index'].values[0]
#chech
f_match_idx

68

<a name = 'rec'></a>

# Prediction:

In [21]:
# get the most similarity scores between all movies and the movie the user inputed
movie_movies_sim = list(enumerate(similarity_scores[f_match_idx]))
#check
print(movie_movies_sim)


[(0, 0.03357074878067544), (1, 0.05464482792361338), (2, 0.013735500604224321), (3, 0.006468756104392056), (4, 0.032689433100733856), (5, 0.013907256685755468), (6, 0.07692837576335507), (7, 0.23944423963486403), (8, 0.007882387851851008), (9, 0.07599206098164224), (10, 0.07536074882460435), (11, 0.011926069211745289), (12, 0.013707618139948927), (13, 0.012376074925089967), (14, 0.09657127116284187), (15, 0.007286271383816741), (16, 0.22704403782296803), (17, 0.013112928084103852), (18, 0.04140526820609592), (19, 0.07883282546834254), (20, 0.07981173664799913), (21, 0.011266873271064943), (22, 0.006892575895462362), (23, 0.006599097891242657), (24, 0.012665208122549731), (25, 0.0), (26, 0.21566241096831154), (27, 0.03058128209382663), (28, 0.06107440221966536), (29, 0.014046184258938894), (30, 0.08077346594769808), (31, 0.31467052449477506), (32, 0.028782099134267002), (33, 0.1308981094105017), (34, 0.0), (35, 0.035350090674865595), (36, 0.03185325269937554), (37, 0.008024326882532318)

In [22]:
# sort the similarity scores list:
movie_movies_sim_sorted = sorted(movie_movies_sim, key = lambda x: x[1], reverse= True)
#check
print(movie_movies_sim_sorted)

[(68, 0.9999999999999998), (79, 0.40890433998005954), (31, 0.31467052449477506), (7, 0.23944423963486403), (16, 0.22704403782296803), (26, 0.21566241096831154), (85, 0.20615862984665329), (182, 0.1957395613961161), (511, 0.16702973947860683), (3623, 0.16092460881355855), (64, 0.15299924139445137), (203, 0.14818667948665115), (174, 0.14719931209420425), (4401, 0.14505971470107842), (101, 0.14401677581826292), (46, 0.14216268867232235), (169, 0.13809470132249058), (1740, 0.1362438264169076), (94, 0.1361681957902901), (788, 0.13305895074229218), (126, 0.13263982780511066), (131, 0.1313769858600653), (33, 0.1308981094105017), (2487, 0.12309731939910505), (783, 0.12162995562040374), (138, 0.11846458075866885), (2442, 0.11725512335483321), (661, 0.11719294096248457), (607, 0.11387063493435634), (38, 0.11261826904871128), (2651, 0.11218787873732045), (353, 0.11168465127044278), (122, 0.1085029603366125), (1553, 0.10797822171513259), (1451, 0.107849394974707), (242, 0.10630339022327012), (618,

In [23]:
# Recommend ten most similar movies:

print('Suggested Movies: \n')

n_mov = 1
for i, _ in movie_movies_sim_sorted[1:]:
  if n_mov < 11:
    title = df[df.index ==i]['title'].values[0]
    print(f"{n_mov}. {title}")
    n_mov +=1


Suggested Movies: 

1. Iron Man 2
2. Iron Man 3
3. Avengers: Age of Ultron
4. The Avengers
5. Captain America: Civil War
6. Captain America: The Winter Soldier
7. Ant-Man
8. X-Men
9. Made
10. X-Men: Apocalypse
